In [11]:
import pandas as pd
import gzip
from tqdm import tqdm
import os
from Bio.Seq import Seq
import csv

In [2]:
def open_fasta(file_path):
    """Open a FASTA file, handling both gzipped and uncompressed files."""
    if file_path.endswith(".gz"):
        return gzip.open(file_path, "rt")  # Open as text mode
    else:
        return open(file_path, "r")  # Open normally

In [3]:
def analyze_orf(seq):

    # Define start and stop codons
    START_CODON = "ATG"
    STOP_CODONS = {"TAA", "TAG", "TGA"}

    seq = str(seq)  # Ensure it's a string

    has_start = seq.startswith(START_CODON)
    has_stop = seq[-3:] in STOP_CODONS if len(seq) >= 3 else False

    # If a start codon is found
    if has_start:
        # Check for internal stop codons in the correct reading frame (starting from 3rd position)
        internal_stop = any(seq[i:i+3] in STOP_CODONS for i in range(3, len(seq) - 6, 3))

        return {
            "start_codon": True,
            "ends_with_stop": has_stop,  # Still check if last 3 bases are a stop codon
            "internal_stop_codons": internal_stop,
            "inferred_reading_frame": 1
        }

    # If no start codon but has a stop codon at the end
    elif has_stop:
        for frame in range(3):
            frame_seq = seq[frame:-3]  # Shift reading frame
            codons = [frame_seq[i:i+3] for i in range(0, len(frame_seq) - 2, 3)]
            if all(codon not in STOP_CODONS for codon in codons):  # No stop codons in frame
                return {
                    "start_codon": False,
                    "ends_with_stop": has_stop,
                    "internal_stop_codons": False,
                    "inferred_reading_frame": frame +1
                }

        return {
            "start_codon": False,
            "ends_with_stop": False,
            "internal_stop_codons": "Cannot determine (no valid ORF found)",
            "inferred_reading_frame": 'N/A'
        }

    # If no start and no stop, try finding a reading frame without internal stops
    else:
        for frame in range(3):
            frame_seq = seq[frame:]  # Shift reading frame
            codons = [frame_seq[i:i+3] for i in range(0, len(frame_seq) - 2, 3)]
            if all(codon not in STOP_CODONS for codon in codons):  # No stop codons in frame
                return {
                    "start_codon": False,
                    "ends_with_stop": False,
                    "internal_stop_codons": False,
                    "inferred_reading_frame": frame +1
                }

        return {
            "start_codon": False,
            "ends_with_stop": False,
            "internal_stop_codons": "Cannot determine (no valid ORF found)",
            "inferred_reading_frame": 'N/A'
        }

In [4]:
hamr_report = "test_data/genomes_hamr.tsv"

In [5]:
hamr_df = pd.read_csv(hamr_report, sep="\t")
hamr_df.head()

,input_file_name,gene_symbol,gene_name,reference_database_name,reference_database_version,reference_accession,analysis_software_name,analysis_software_version,genetic_variation_type,antimicrobial_agent,...,amino_acid_mutation_interpretation,reference_gene_length,reference_gene_start,reference_gene_stop,reference_protein_length,reference_protein_start,reference_protein_stop,resistance_mechanism,strand_orientation,sequence_identity
0,Col_1,blaDHA,class C beta-lactamase DHA-12,NCBI Reference Gene Database,2024-12-18.1,WP_025151790.1,amrfinderplus,4.0.19,gene_presence_detected,CEPHALOSPORIN,...,NaN,NaN,NaN,NaN,379.0,NaN,NaN,NaN,-,98.94
1,Col_1,catA2,type A-2 chloramphenicol O-acetyltransferase C...,NCBI Reference Gene Database,2024-12-18.1,WP_011751353.1,amrfinderplus,4.0.19,gene_presence_detected,CHLORAMPHENICOL,...,NaN,NaN,NaN,NaN,213.0,NaN,NaN,NaN,+,94.37
2,Col_1,tet(D),tetracycline efflux MFS transporter Tet(D),NCBI Reference Gene Database,2024-12-18.1,WP_001039466.1,amrfinderplus,4.0.19,gene_presence_detected,TETRACYCLINE,...,NaN,NaN,NaN,NaN,394.0,NaN,NaN,NaN,-,96.45
3,Col_1,ArnT,pmr phosphoethanolamine transferase,CARD,Matplotlib,3005053,rgi,Matplotlib,gene_presence_detected,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,antibiotic target alteration,+,46.74
4,Col_1,ArnT,pmr phosphoethanolamine transferase,CARD,Matplotlib,3005053,rgi,Matplotlib,gene_presence_detected,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,antibiotic target alteration,+,45.37


In [14]:
input_fasta = 'test_data/all_genomes.fasta'
output_csv = 'test_data/Genomes_PsuedoAMR_Oout.csv'

name_breaks = ['_allclass', '_scaffolds']

seqs = set(hamr_df['input_sequence_id'])

seqs.update(hamr_df['input_sequence_id'].apply(lambda x: x.rsplit("_", 1)[0]))

total_size = os.path.getsize(input_fasta)

with open_fasta(input_fasta) as infile, open(output_csv, "w", newline="") as outfile, tqdm(
        total=total_size, unit="B", unit_scale=True, desc="Processing Fasta - This bar is probably inaccurate:" # setting up the progess bar, dose net work well, but at least shows its running....
    ) as pbar:
        writer = csv.writer(outfile, quoting=csv.QUOTE_MINIMAL)
        writer.writerow([
            "samplename", "tool", "produced_orf", "gene_symbol", "coverage", "sequence_identity",
            "3'Start", "5'Stop", "3'_at_seq_end", "start_pos", "5'_at_seq_end", "end_pos",
            "total_len_queryseq", "sense", "Internal_stops", "Reading_Frame",
            "antimicrobial_agent", "drug_class", "input_sequence_id",
            "Reported_Sequence", "ORF", "Protein"
        ])
        while True:
            pos = infile.tell()  # Get current file position for progress tracking
            header = infile.readline().strip()
            if not header:
                break  # End of file

            sequence = infile.readline().strip()

            read_name  = header[1:]  # Extract read identifier


            if read_name in seqs:
                subset_df = hamr_df[hamr_df['input_sequence_id'].apply(lambda x: x.startswith(read_name))]
                for _, row in subset_df.iterrows():
                    start = int(row['input_gene_start']) - 1
                    stop = int(row['input_gene_stop']) - 1

                    if row['strand_orientation'] == '-':
                        current_cd = Seq(sequence[start:stop]).reverse_complement()
                        seq_len = len(str(sequence))
                        if start <= 2:
                            Read_end = True
                        else:
                            Read_end = False

                        if int(seq_len) - stop <= 3:
                            Read_start = True
                        else:
                            Read_start = False

                    else:
                        current_cd = Seq(sequence[start:stop])
                        seq_len = len(str(sequence))
                        if start <= 2:
                            Read_start = True
                        else:
                            Read_start = False

                        if int(seq_len) - stop <= 3:
                            Read_end = True
                        else:
                            Read_end = False

                    result = analyze_orf(current_cd)

                    gene_symbol = row['gene_symbol']
                    cov = row['coverage_percentage']

                    if result['inferred_reading_frame'] in [1, 2, 3]:
                        #Get frame
                        frame = int(result['inferred_reading_frame']) -1
                        # Start at the third base (index 2)
                        cd_seq = str(current_cd)[frame:]

                        # Ensure the length is a multiple of 3
                        length_to_cut = len(cd_seq) % 3  # Calculate the remainder when divided by 3
                        if length_to_cut != 0:
                            cd_seq = cd_seq[:-length_to_cut]  # Remove the excess bases to make it a multiple of 3
                    else:
                        cd_seq = 'N/A'

                    if cd_seq != 'N/A':
                        # Create a Seq object
                        dna_seq = Seq(cd_seq)
                        real_orf = True

                        # Translate the DNA sequence into a protein sequence
                        protein_seq = dna_seq.translate()
                    else:
                        protein_seq = 'N/A'
                        real_orf = False

                    if cd_seq != 'N/A' and result['internal_stop_codons'] == False:
                        real_orf = True
                    else:
                        real_orf = False

                    sample_name = row['input_file_name']

                    for break_str in name_breaks:
                        if sample_name.endswith(break_str):
                            sample_name = sample_name[:-len(break_str)]  # Remove the suffix

                    tool = row['reference_database_name']

                    sequence_identity = row['sequence_identity']
                    antimicrobial_agent = row['antimicrobial_agent']
                    drug_class = row['drug_class']



                    writer.writerow([
                        sample_name, tool, real_orf, gene_symbol, cov, sequence_identity,
                        result['start_codon'], result['ends_with_stop'], Read_start, start + 1,
                        Read_end, stop + 1, len(sequence), row['strand_orientation'],
                        result['internal_stop_codons'], result['inferred_reading_frame'],
                        antimicrobial_agent, drug_class, read_name, str(current_cd), cd_seq, str(protein_seq)
                    ])

            # Update progress
            if input_fasta.endswith(".gz"):
                pbar.update((len(header) + len(sequence)) / 6)  # Approximate uncompressed progress
            else:
                pbar.update(infile.tell() - pos)  # Accurate for uncompressed files

Processing Fasta - This bar is probably inaccurate:: 100%|██████████| 366M/366M [00:02<00:00, 171MB/s] 


In [15]:
ARG_DB = pd.read_csv(output_csv)
ARG_DB.head()

,samplename,tool,produced_orf,gene_symbol,coverage,sequence_identity,3'Start,5'Stop,3'_at_seq_end,start_pos,...,total_len_queryseq,sense,Internal_stops,Reading_Frame,antimicrobial_agent,drug_class,input_sequence_id,Reported_Sequence,ORF,Protein
0,Col_10,NCBI Reference Gene Database,True,aph(3'')-Ib,100.0,100.0,False,False,False,974,...,15181,+,False,1.0,STREPTOMYCIN,AMINOGLYCOSIDE,PLAS_P:0.998|C:0.002_NODE_42_length_15181_cov_...,TTGAATCGAACTAATATTTTTTTTGGTGAATCGCATTCTGACTGGT...,TTGAATCGAACTAATATTTTTTTTGGTGAATCGCATTCTGACTGGT...,LNRTNIFFGESHSDWLPVRGGESGDFVFRRGDGHAFAKIAPASRRG...
1,Col_10,NCBI Reference Gene Database,True,aph(6)-Id,100.0,100.0,True,False,False,1777,...,15181,+,False,1.0,STREPTOMYCIN,AMINOGLYCOSIDE,PLAS_P:0.998|C:0.002_NODE_42_length_15181_cov_...,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...,MFMPPVFPAHWHVSQPVLIADTFSSLVWKVSLPDGTPAIVKGLKPI...
2,Col_10,NCBI Reference Gene Database,True,floR,100.0,99.5,False,False,False,5751,...,15181,-,False,3.0,CHLORAMPHENICOL/FLORFENICOL,PHENICOL,PLAS_P:0.998|C:0.002_NODE_42_length_15181_cov_...,TGACCACCACACGCCCCGCGTGGGCCTATACGCTGCCGGCAGCACT...,ACCACCACACGCCCCGCGTGGGCCTATACGCTGCCGGCAGCACTGC...,TTTRPAWAYTLPAALLLMAPFDILASLAMDIYLPVVPAMPGILNTT...
3,Col_10,NCBI Reference Gene Database,True,sul2,100.0,100.0,True,False,False,98,...,15181,+,False,1.0,SULFONAMIDE,SULFONAMIDE,PLAS_P:0.998|C:0.002_NODE_42_length_15181_cov_...,ATGAATAAATCGCTCATCATTTTCGGCATCGTCAACATAACCTCGG...,ATGAATAAATCGCTCATCATTTTCGGCATCGTCAACATAACCTCGG...,MNKSLIIFGIVNITSDSFSDGGRYLAPDAAIAQARKLMAEGADVID...
4,Col_10,NCBI Reference Gene Database,True,tet(A),100.0,100.0,False,False,False,3949,...,15181,+,False,1.0,TETRACYCLINE,TETRACYCLINE,PLAS_P:0.998|C:0.002_NODE_42_length_15181_cov_...,GTGAAACCCAACAGACCCCTGATCGTAATTCTGAGCACTGTCGCGC...,GTGAAACCCAACAGACCCCTGATCGTAATTCTGAGCACTGTCGCGC...,VKPNRPLIVILSTVALDAVGIGLIMPVLPGLLRDLVHSNDVTAHYG...


In [17]:
ARG_DB = ARG_DB.drop_duplicates()
ARG_DB = ARG_DB.loc[ARG_DB['produced_orf'] == True]
ARG_DB = ARG_DB.loc[ARG_DB['coverage'] >= 90]
ARG_DB.to_csv('./test_data/genomes_filtered.csv', index=False)